# Reconciliation #04: Claim Development Patterns

## Overview

This notebook verifies that claims develop (pay out) according to their configured
actuarial patterns, that outstanding liabilities decrease at the correct rate, and
that the sum of all payments equals the original incurred amount.

We test all four built-in development patterns:

| Pattern | Duration | Typical Line of Business |
|---|---|---|
| Immediate | 1 year | Property / equipment damage |
| Medium-tail (5yr) | 5 years | Workers compensation |
| Long-tail (10yr) | 10 years | General liability |
| Very-long-tail (15yr) | 15 years | Product liability |

For each pattern we verify:
1. Development factors sum to 1.0
2. Year-by-year payments match the expected factor schedule
3. Cumulative paid converges to 100% of incurred
4. `Sum(all payments) == Original Incurred Amount` (tolerance < $0.01)
5. Outstanding liability at each step == Incurred - Cumulative Paid

**Prerequisites:** `ergodic_insurance` package installed (`pip install ergodic-insurance`)

**Target runtime:** < 60 seconds

**Audience:** Actuaries, developers, QA

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/reconciliation'

    if os.path.exists(os.path.join(NOTEBOOK_DIR, '_reconciliation_helpers.py')):
        print(f"Found helper module in {NOTEBOOK_DIR}")
        os.chdir(NOTEBOOK_DIR)
        if NOTEBOOK_DIR not in sys.path:
            sys.path.append(NOTEBOOK_DIR)
    else:
        print(f"WARNING: _reconciliation_helpers.py not found in {NOTEBOOK_DIR}")
        print("Please update NOTEBOOK_DIR in this cell to the correct folder path.")

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
# Setup: imports and helpers
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure the reconciliation helpers are importable
sys.path.insert(0, os.path.dirname(os.path.abspath(".")))
sys.path.insert(0, os.path.abspath("."))

from _reconciliation_helpers import (
    ReconciliationChecker,
    final_summary,
    section_header,
    notebook_header,
    timed_cell,
    fmt_dollar,
    display_df,
)

from ergodic_insurance.claim_development import ClaimDevelopment
from ergodic_insurance.claim_liability import ClaimLiability

# Fixed random seed for reproducibility
np.random.seed(42)

# Display the notebook header
notebook_header(
    number=4,
    title="Claim Development Patterns",
    description=(
        "Verify that claims pay out according to their actuarial development "
        "patterns, outstanding liabilities decrease correctly, and total "
        "payments equal the original incurred amount."
    ),
)

In [ ]:
# Configuration constants
CLAIM_AMOUNT = 1_000_000.0  # $1M test claim
ACCIDENT_YEAR = 2020
TOLERANCE = 0.01  # $0.01

# All built-in patterns to test
PATTERNS = {
    "Immediate": ClaimDevelopment.create_immediate(),
    "Medium-Tail (5yr)": ClaimDevelopment.create_medium_tail_5yr(),
    "Long-Tail (10yr)": ClaimDevelopment.create_long_tail_10yr(),
    "Very-Long-Tail (15yr)": ClaimDevelopment.create_very_long_tail_15yr(),
}

# Collect checkers for final summary
all_checkers = []

print(f"Test claim amount: {fmt_dollar(CLAIM_AMOUNT)}")
print(f"Accident year: {ACCIDENT_YEAR}")
print(f"Tolerance: ${TOLERANCE}")
print(f"Patterns under test: {', '.join(PATTERNS.keys())}")

---

## Helper Function: Simulate and Verify a Development Pattern

This function drives the full test sequence for any development pattern:
display the factor table, simulate the payment schedule, plot cumulative
paid vs. incurred, and run all reconciliation checks.

In [ ]:
def test_development_pattern(
    pattern_label: str,
    pattern: ClaimDevelopment,
    claim_amount: float,
    accident_year: int,
    tolerance: float,
) -> ReconciliationChecker:
    """Simulate and verify a single development pattern.

    Returns a ReconciliationChecker with all check results.
    """
    checker = ReconciliationChecker(section=pattern_label)
    n_years = len(pattern.development_factors)

    # --- 1. Development Factor Table ---
    factor_data = {
        "Year": list(range(1, n_years + 1)),
        "Incremental %": [f"{f * 100:.1f}%" for f in pattern.development_factors],
        "Incremental $": [fmt_dollar(f * claim_amount) for f in pattern.development_factors],
        "Cumulative %": [
            f"{pattern.get_cumulative_paid(y) * 100:.1f}%" for y in range(1, n_years + 1)
        ],
    }
    factor_df = pd.DataFrame(factor_data)
    display_df(factor_df, title=f"{pattern_label} - Development Factor Table")

    # Check: factors sum to 1.0
    factor_sum = sum(pattern.development_factors) + pattern.tail_factor
    checker.assert_close(
        factor_sum,
        1.0,
        tol=0.01,
        message=f"Development factors sum to 1.0",
        label_actual="Sum(factors)",
        label_expected="1.0",
    )

    # --- 2. Simulate Payment Schedule Year by Year ---
    # Extend simulation by 2 extra years beyond the pattern to ensure no extra payments
    max_sim_years = n_years + 2
    years = []
    incremental_payments = []
    cumulative_payments = []
    outstanding_liabilities = []
    cumulative_pct = []
    running_cumulative = 0.0

    for dev_year in range(max_sim_years):
        payment_year = accident_year + dev_year
        payment = pattern.calculate_payments(claim_amount, accident_year, payment_year)
        running_cumulative += payment
        outstanding = claim_amount - running_cumulative

        years.append(payment_year)
        incremental_payments.append(payment)
        cumulative_payments.append(running_cumulative)
        outstanding_liabilities.append(outstanding)
        cumulative_pct.append(running_cumulative / claim_amount * 100)

    # Build payment schedule DataFrame
    schedule_df = pd.DataFrame({
        "Payment Year": years,
        "Dev Year": list(range(max_sim_years)),
        "Incremental Payment": [fmt_dollar(p) for p in incremental_payments],
        "Cumulative Paid": [fmt_dollar(c) for c in cumulative_payments],
        "Outstanding": [fmt_dollar(o) for o in outstanding_liabilities],
        "Cumulative %": [f"{p:.1f}%" for p in cumulative_pct],
    })
    display_df(schedule_df, title=f"{pattern_label} - Payment Schedule")

    # --- 3. Plot: Cumulative Paid vs. Original Incurred ---
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(years, cumulative_pct, "o-", color="#4a86c8", linewidth=2, markersize=5, label="Cumulative Paid %")
    ax.axhline(y=100.0, color="#dc3545", linestyle="--", linewidth=1.5, label="100% Incurred")
    ax.set_xlabel("Payment Year")
    ax.set_ylabel("Cumulative Paid (% of Incurred)")
    ax.set_title(f"{pattern_label}: Cumulative Paid vs. Original Incurred")
    ax.set_ylim(-5, 115)
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # --- 4. Verify: Sum(all payments) == Original Incurred Amount ---
    total_paid = sum(incremental_payments)
    checker.assert_close(
        total_paid,
        claim_amount,
        tol=tolerance,
        message=f"Sum(all payments) == Incurred ({fmt_dollar(claim_amount)})",
        label_actual="Total Paid",
        label_expected="Incurred",
    )

    # --- 5. Verify: Outstanding at each step == Incurred - Cumulative Paid ---
    all_outstanding_correct = True
    for i, (cum_paid, outstanding) in enumerate(zip(cumulative_payments, outstanding_liabilities)):
        expected_outstanding = claim_amount - cum_paid
        if abs(outstanding - expected_outstanding) > tolerance:
            all_outstanding_correct = False
            break

    checker.check(
        all_outstanding_correct,
        "Outstanding == Incurred - Cumulative Paid (all years)",
        detail=f"Checked {max_sim_years} development years",
    )

    # --- 6. Verify: get_cumulative_paid() consistency ---
    cumulative_api_consistent = True
    for dev_year in range(1, n_years + 1):
        api_cum_pct = pattern.get_cumulative_paid(dev_year)
        # Manually compute expected cumulative from factors
        manual_cum_pct = sum(pattern.development_factors[:dev_year])
        if abs(api_cum_pct - manual_cum_pct) > 1e-10:
            cumulative_api_consistent = False
            break

    checker.check(
        cumulative_api_consistent,
        "get_cumulative_paid() matches manual sum of factors",
        detail=f"Checked {n_years} development ages",
    )

    # --- 7. Verify: No payments beyond pattern period ---
    payment_beyond = pattern.calculate_payments(
        claim_amount, accident_year, accident_year + n_years + 5
    )
    checker.assert_close(
        payment_beyond,
        0.0,
        tol=tolerance,
        message="No payment beyond development period",
        label_actual="Payment at year+" + str(n_years + 5),
        label_expected="$0",
    )

    # --- 8. Verify: No payment before accident year ---
    payment_before = pattern.calculate_payments(
        claim_amount, accident_year, accident_year - 1
    )
    checker.assert_close(
        payment_before,
        0.0,
        tol=tolerance,
        message="No payment before accident year",
        label_actual="Payment at year-1",
        label_expected="$0",
    )

    # Display results
    checker.display_results()

    return checker

---

## Section 1: Immediate Payment Pattern

Property/equipment damage claims that pay 100% in the first year.

In [ ]:
section_header("1. Immediate Payment Pattern (Property Damage)")

with timed_cell("Immediate Pattern"):
    checker_immediate = test_development_pattern(
        pattern_label="Immediate",
        pattern=PATTERNS["Immediate"],
        claim_amount=CLAIM_AMOUNT,
        accident_year=ACCIDENT_YEAR,
        tolerance=TOLERANCE,
    )
    all_checkers.append(checker_immediate)

---

## Section 2: Medium-Tail 5-Year Pattern

Workers compensation claims developing over 5 years: [40%, 25%, 15%, 10%, 10%].

In [ ]:
section_header("2. Medium-Tail 5-Year Pattern (Workers Comp)")

with timed_cell("Medium-Tail 5yr"):
    checker_medium = test_development_pattern(
        pattern_label="Medium-Tail (5yr)",
        pattern=PATTERNS["Medium-Tail (5yr)"],
        claim_amount=CLAIM_AMOUNT,
        accident_year=ACCIDENT_YEAR,
        tolerance=TOLERANCE,
    )
    all_checkers.append(checker_medium)

---

## Section 3: Long-Tail 10-Year Pattern

General liability claims developing over 10 years: [10%, 20%, 20%, 15%, 10%, 8%, 7%, 5%, 3%, 2%].

In [ ]:
section_header("3. Long-Tail 10-Year Pattern (General Liability)")

with timed_cell("Long-Tail 10yr"):
    checker_long = test_development_pattern(
        pattern_label="Long-Tail (10yr)",
        pattern=PATTERNS["Long-Tail (10yr)"],
        claim_amount=CLAIM_AMOUNT,
        accident_year=ACCIDENT_YEAR,
        tolerance=TOLERANCE,
    )
    all_checkers.append(checker_long)

---

## Section 4: Very-Long-Tail 15-Year Pattern

Product liability claims developing over 15 years: [5%, 10%, 15%, 15%, 12%, 10%, 8%, 6%, 5%, 4%, 3%, 3%, 2%, 1%, 1%].

In [ ]:
section_header("4. Very-Long-Tail 15-Year Pattern (Product Liability)")

with timed_cell("Very-Long-Tail 15yr"):
    checker_very_long = test_development_pattern(
        pattern_label="Very-Long-Tail (15yr)",
        pattern=PATTERNS["Very-Long-Tail (15yr)"],
        claim_amount=CLAIM_AMOUNT,
        accident_year=ACCIDENT_YEAR,
        tolerance=TOLERANCE,
    )
    all_checkers.append(checker_very_long)

---

## Section 5: ClaimLiability Integration Test

Verify that `ClaimLiability` (which uses `ClaimDevelopment` as a strategy)
produces consistent results: `get_payment()` + `make_payment()` should drain
the remaining balance to zero.

In [ ]:
section_header("5. ClaimLiability Integration")

checker_liability = ReconciliationChecker(section="ClaimLiability Integration")

with timed_cell("ClaimLiability Integration"):
    for label, pattern in PATTERNS.items():
        liability = ClaimLiability(
            original_amount=CLAIM_AMOUNT,
            remaining_amount=CLAIM_AMOUNT,
            year_incurred=ACCIDENT_YEAR,
            development_strategy=pattern,
        )

        n_years = len(pattern.development_factors)
        total_paid = 0.0

        for yr in range(n_years):
            scheduled = liability.get_payment(yr)
            actual = liability.make_payment(scheduled)
            total_paid += float(actual)

        # Remaining should be ~0
        remaining = float(liability.remaining_amount)
        checker_liability.assert_close(
            remaining,
            0.0,
            tol=TOLERANCE,
            message=f"{label}: remaining == $0 after full development",
            label_actual="Remaining",
            label_expected="$0",
        )

        # Total paid should equal original
        checker_liability.assert_close(
            total_paid,
            CLAIM_AMOUNT,
            tol=TOLERANCE,
            message=f"{label}: total paid == original amount",
            label_actual="Total Paid",
            label_expected="Incurred",
        )

    checker_liability.display_results()
    all_checkers.append(checker_liability)

---

## Section 6: Cross-Pattern Comparison

Compare cumulative paid curves across all four patterns on a single chart.
Verify that longer-tail patterns develop more slowly but all converge to 100%.

In [ ]:
section_header("6. Cross-Pattern Comparison")

checker_comparison = ReconciliationChecker(section="Cross-Pattern Comparison")

with timed_cell("Cross-Pattern Comparison"):
    fig, ax = plt.subplots(figsize=(12, 6))
    colors = ["#28a745", "#4a86c8", "#ff7f0e", "#dc3545"]

    # For ordering check: at year 1, longer-tail should have lower cumulative
    year1_pcts = {}

    for (label, pattern), color in zip(PATTERNS.items(), colors):
        n_years = len(pattern.development_factors)
        max_plot_years = max(n_years, 15) + 1
        dev_years = list(range(0, max_plot_years + 1))
        cum_pct = [pattern.get_cumulative_paid(y) * 100 for y in dev_years]

        ax.plot(
            dev_years, cum_pct, "o-",
            color=color, linewidth=2, markersize=4,
            label=f"{label} ({n_years}yr)",
        )

        year1_pcts[label] = pattern.get_cumulative_paid(1)

        # All patterns should reach 100% eventually
        final_cum = pattern.get_cumulative_paid(n_years)
        checker_comparison.assert_close(
            final_cum,
            1.0,
            tol=0.01,
            message=f"{label}: reaches 100% at maturity",
            label_actual=f"Cum% at year {n_years}",
            label_expected="100%",
        )

    ax.axhline(y=100.0, color="gray", linestyle="--", linewidth=1, alpha=0.6)
    ax.set_xlabel("Development Year")
    ax.set_ylabel("Cumulative Paid (%)")
    ax.set_title("Claim Development Patterns: Cumulative Paid Comparison")
    ax.set_ylim(-5, 115)
    ax.legend(loc="lower right")
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Ordering checks: immediate should be fastest, very-long-tail slowest
    checker_comparison.assert_greater(
        year1_pcts["Immediate"],
        year1_pcts["Medium-Tail (5yr)"],
        message="Immediate develops faster than Medium-Tail at year 1",
    )
    checker_comparison.assert_greater(
        year1_pcts["Medium-Tail (5yr)"],
        year1_pcts["Long-Tail (10yr)"],
        message="Medium-Tail develops faster than Long-Tail at year 1",
    )
    checker_comparison.assert_greater(
        year1_pcts["Long-Tail (10yr)"],
        year1_pcts["Very-Long-Tail (15yr)"],
        message="Long-Tail develops faster than Very-Long-Tail at year 1",
    )

    # Summary table of year-1 development percentages
    summary_data = {
        "Pattern": list(PATTERNS.keys()),
        "Duration (yrs)": [len(p.development_factors) for p in PATTERNS.values()],
        "Year 1 Paid %": [f"{year1_pcts[k] * 100:.1f}%" for k in PATTERNS.keys()],
        "Factors Sum": [
            f"{sum(p.development_factors) + p.tail_factor:.4f}"
            for p in PATTERNS.values()
        ],
    }
    display_df(pd.DataFrame(summary_data), title="Pattern Summary")

    checker_comparison.display_results()
    all_checkers.append(checker_comparison)

---

## Section 7: Multiple Claim Amounts

Verify that development patterns scale linearly with claim amount.
Test with small, medium, and large claims across all patterns.

In [ ]:
section_header("7. Scaling Verification (Multiple Claim Amounts)")

checker_scaling = ReconciliationChecker(section="Scaling Verification")

with timed_cell("Scaling Verification"):
    test_amounts = [100.0, 50_000.0, 1_000_000.0, 25_000_000.0]

    for pattern_label, pattern in PATTERNS.items():
        n_years = len(pattern.development_factors)

        for amount in test_amounts:
            total_paid = sum(
                pattern.calculate_payments(amount, ACCIDENT_YEAR, ACCIDENT_YEAR + y)
                for y in range(n_years + 5)
            )
            checker_scaling.assert_close(
                total_paid,
                amount,
                tol=TOLERANCE,
                message=f"{pattern_label} @ {fmt_dollar(amount)}: total == incurred",
                label_actual="Total Paid",
                label_expected="Incurred",
            )

    checker_scaling.display_results()
    all_checkers.append(checker_scaling)

---

## Final Summary

In [ ]:
section_header("Final Summary")
final_summary(*all_checkers)